### Requirements

- langchain>=0.0.200
- openai>=0.27.8
- chromadb>=0.4.7
- beautifulsoup4>=4.12.2
- selenium>=4.11.2
- webdriver-manager>=3.8.6
- python-dotenv>=1.0.0
- pprintpp>=0.4.0

## 트러블 슈팅

> 2024/11/22 (성진)
---
**데이터 전처리 과정**
1. WebBaseLoader 로 레퍼런스가 정상적으로 인식되지 않는 문제  
 - 원인 : WebBaseLoader의 한계로 인한 실패 가능성. WebBaseLoader는 간단한 텍스트 추출에 적합하며, 복잡한 HTML 구조나 JavaScript 기반의 동적 콘텐츠를 처리하지 못할 수 있다. 

    1_1. WebBaseLoader 대신 request를 사용한 HTML 크롤링 시도 -> 실패  
     - 원인 : Notion 페이지는 JavaScript를 활용한 동적 렌더링을 사용하는 구조여서 requests만 사용해서는 JavaScript로 렌더링된 내용을 가져올 수 없었다.

    1_2. Selenium 사용 --> 부분 성공  
     - 원인 : implicitly_wait 으로 대기 후 로드 시 데이터를 가져오는 경우가 있고, 못가져오는 경우가 있었음..

    1_3. 대기 방식 변경 (WebDriverWait.until() 사용) --> 성공  
     - 로드하려는 웹페이지의 html 중 특정 요소가 로드될 때까지 wait하는 방식 사용

2. soup.get_text() 에서 노션 문서의 헤드라인만 추출되는 문제  
 - 원인 : 노션 페이지에 진입했을 때, 토글(Ctrl+Alt+T)이 닫힌 상태로 텍스트가 추출되고 있었음

    2_1. 단축키 입력을 선언하는 방법 (헤드리스 상태에서 사용 불가)  

    2_2. html 내에 토글 버튼을 찾아 여는 방법 (헤드리스 상태에서 사용 가능) --> 선택 (성공)  
     - drive.find_elements 로 버튼 탐색 후 클릭 (지연시간 1초)

3. 추출된 텍스트 파일에 일부 문법 구문이 남아있는 문제  
 - re (regular expression) 을 활용해 삭제/전처리 (성공)

**유사한 질문을 계속해서 질문하는 경우**
1. cosine 유사도 비교로 유사한 질문을 생성해내는 경우에는 다른 질문을 생성하도록 했음
   - cosine 유사도는 단어의 유사도를 비교하는 것에 약점을 보였기 때문에 성능이 크게 좋아지지 않음 (실패)
   - 프롬프팅으로 고도화 시도 (성공))


---
**AI 챗봇**

1. AI 가 질문에 대한 사용자의 답변을 인식하지 못하는 문제  
 - 사용자의 답변을 질문과 함께 feedback_prompt 형식으로 묶어서 다시 AI에게 전달하는 방식으로 문제 해결

2. AI 가 동일한 주제에 대해서 반복적으로 비슷한 질문만을 하는 문제 --> (해결중..)

---

> 2024/11/26 (수연)

2. AI 가 동일한 주제에 대해서 반복적으로 비슷한 질문만을 하는 문제 -> 해결

   - 해결 방법 : 
   - 수동적으로 참고할 docs와 갯수를 할당 (코드에서는 한 교재당 2개의 질문)
   - 유사한 질문을 rag에 넘겨주어 겹치지 않도록 프롬프팅 (직전 5개의 문장 기준, 이미 한 docs당 2개의 문장 질문 제한 고려)
   - 프롬프팅 고도화


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

from dotenv import load_dotenv
import os

from selenium.webdriver.common.keys import Keys

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
def save_file(txt:str, file_name:str):

    with open(file_name, 'w', encoding='utf-8') as content_file:
        content_file.write(txt)

    print(f"TEXT 파일 저장 완료: {file_name}")

In [4]:

# .env 파일에서 환경변수 로드
load_dotenv()
api_key = os.getenv("OPEN_AI_KEY")

llm = ChatOpenAI(model="gpt-4o", api_key=api_key)

### 교재 저장하는 부분

In [ ]:

# Selenium 옵션 설정 (헤드리스 모드로 실행)
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
chrome_options.add_argument("--disable-gpu")  # GPU 비활성화 (일부 환경에서 필요)

# WebDriver 경로 설정 (자동 설치)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url_list=[]
txt_list=[]

# 환경변수에 저장된 URL 로드
for i in range(1, 17):  # URL_1 ~ URL_16
    url = os.getenv(f"DL_URL_{i}")
    if url:  # 환경변수가 존재하면 추가
        url_list.append(url)

j = 0
# 웹페이지 요청
for url in url_list:
    j+= 1
    txt = ""
    driver.get(url)  # 페이지 로드

    # 특정 요소가 로드될 때까지 기다림 (예: Notion 페이지에서 주요 콘텐츠가 담길 요소)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".notion-page-content"))
        )
    except TimeoutException:
        print(f"페이지 로딩 실패: {url}")
        continue
    
    # 토글이 닫혀 있으면 토글을 열기
    try:
        # 모든 토글 버튼을 찾음 (Ctrl+Alt+T에 해당하는 토글을 찾아서 열기)
        toggle_buttons = driver.find_elements(By.XPATH, "//div[@role='button' and (@aria-expanded='false')]")
        print(len(toggle_buttons))
        
        # 각 토글을 클릭하여 열기
        for button in toggle_buttons:
            driver.execute_script("arguments[0].click();", button)
            time.sleep(1)  # 토글이 열리기 전에 잠깐 대기
        
    except Exception as e:
        print(f"토글을 여는 데 실패했습니다: {e}")

    # 페이지의 HTML 가져오기
    html_code = driver.page_source

    # BeautifulSoup으로 HTML 파싱
    soup = BeautifulSoup(html_code, 'html.parser')

    txt = soup.get_text()

    # 1. \xa0를 공백으로 변환
    txt = txt.replace('\xa0', ' ')

    # 2. 정규식을 사용해 \\로 시작하는 LaTeX 명령어 제거
    txt = re.sub(r'\\[a-zA-Z]+\{.*?\}', '', txt)  # \command{...} 형식 제거
    txt = re.sub(r'\\[a-zA-Z]+', '', txt)        # \command 형식 제거
    txt = re.sub(r'💡모든 토글을 열고 닫는 단축키\nWindows : [^\n]*\nMac : [^\n]*\n', '', txt)

    # 3. 불필요한 공백 제거 (코드 개행 유지를 위해 주석처리)
    # txt = re.sub(r'\s+', ' ', txt).strip()

    # 텍스트만 가져오기
    txt_list.append(txt)
    save_file(''.join(txt), f"DL_dataset_{j}.txt")
    


driver.quit()  # 브라우저 종료


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

4
TEXT 파일 저장 완료: DL_dataset_1.txt
6
TEXT 파일 저장 완료: DL_dataset_2.txt
5
TEXT 파일 저장 완료: DL_dataset_3.txt
3
TEXT 파일 저장 완료: DL_dataset_4.txt
5
TEXT 파일 저장 완료: DL_dataset_5.txt
4
TEXT 파일 저장 완료: DL_dataset_6.txt
3
TEXT 파일 저장 완료: DL_dataset_7.txt
4
TEXT 파일 저장 완료: DL_dataset_8.txt
3
TEXT 파일 저장 완료: DL_dataset_9.txt
3
TEXT 파일 저장 완료: DL_dataset_10.txt
3
TEXT 파일 저장 완료: DL_dataset_11.txt
4
TEXT 파일 저장 완료: DL_dataset_12.txt
2
TEXT 파일 저장 완료: DL_dataset_13.txt
4
TEXT 파일 저장 완료: DL_dataset_14.txt
2
TEXT 파일 저장 완료: DL_dataset_15.txt
3
TEXT 파일 저장 완료: DL_dataset_16.txt


In [ ]:
ml_retriever_list = []
dl_retriever_list = []
llm_retriever_list = []
python_retriever_list = []
open_source_retrievver_list = []

### 저장한 교재 (txt) 불러오는 코드

In [13]:
import os
text_list = []

# 파일을 읽어와서 text_list에 저장하는 함수
def load_files_to_list(file_path, text_list):
    if os.path.exists(file_path):  # 파일이 존재하는지 확인
        with open(file_path, 'r', encoding='utf-8') as file:  # 파일 열기
            content = file.read()  # 파일 내용 읽기
            text_list.append(content)  # text_list에 추가
    else:
        print(f"File {file_path} not found.")
        
    

# 예시: 파일 경로 목록을 지정
file_paths = [f"ML_dataset_{i}.txt" for i in range(1, 17)]  # 파일 경로 목록 (DL_dataset_1.txt, DL_dataset_2.txt, ...)

print(file_paths)

for j in range(len(file_paths)):
    load_files_to_list(file_paths[j], text_list)

text_list


['ML_dataset_1.txt', 'ML_dataset_2.txt', 'ML_dataset_3.txt', 'ML_dataset_4.txt', 'ML_dataset_5.txt', 'ML_dataset_6.txt', 'ML_dataset_7.txt', 'ML_dataset_8.txt', 'ML_dataset_9.txt', 'ML_dataset_10.txt', 'ML_dataset_11.txt', 'ML_dataset_12.txt', 'ML_dataset_13.txt', 'ML_dataset_14.txt', 'ML_dataset_15.txt', 'ML_dataset_16.txt']


['[스파르타코딩클럽] 1강. 강의 소개📘[SCC] 바닥부터 시작하는 머신러닝/📚[스파르타코딩클럽] 바닥부터 시작하는 머신러닝 - 1주차/📕[스파르타코딩클럽] 1강. 강의 소개Made with📕[스파르타코딩클럽] 1강. 강의 소개[수업 목표]강의 소개 및 머신러닝 기본 개념을 알아봅시다[목차]01. 강좌 소개🤔 딥러닝이 대세라던데, 왜 머신러닝부터 시작할까요?02. 머신러닝 소개강의 뒤쪽에서 나오는 어려운 문법들은 강좌 내에서 설명 후에 실습을 진행할 예정이니 걱정하지 않으셔도 됩니다!강의 뒤쪽에서 나오는 어려운 문법들은 강좌 내에서 설명 후에 실습을 진행할 예정이니 걱정하지 않으셔도 됩니다!\ufeff\u200b🤔 딥러닝이 대세라던데, 왜 머신러닝부터 시작할까요?딥러닝의 기초는 탄탄한 머신러닝 지식입니다. 기초를 튼튼히 쌓아야 딥러닝도 쉽게 이해할 수 있죠! 차근차근 기초부터 다지면, 딥러닝 정복도 문제없어요! 💪02. 머신러닝 소개✔️머신러닝에 대한 기본적인 개념 및 역사등을 소개합니다1)머신러닝 소개☑️ 머신러닝의 정의컴퓨터가 명시적으로 프로그래밍 되지 않아도 데이터를 통해 학습하고, 예측할 수 있도록 하는 기능대량의 데이터를 알고리즘에 입력하여 학습과정을 통해 모델을 생성하고 예측을 수행기존 프로그램은 명시적으로 작성된 규칙과 논리에 따라 동작한다는것에서 머신러닝과의 차이가 있습니다!기존 프로그램은 명시적으로 작성된 규칙과 논리에 따라 동작한다는것에서 머신러닝과의 차이가 있습니다!\ufeff\u200b전통적인 프로그래밍:규칙과 논리를 프로그래머가 직접 정의 🖥️명시적 명령과 조건문을 통해 문제 해결 📝머신러닝:데이터를 이용해 패턴과 규칙을 스스로 학습 📊예측 모델을 통해 새로운 데이터에 대한 결과 도출 🔮프로그램이 아닌 모델이 중심 ⚙️☑️ 머신러닝의 구성요소데이터셋 : 모델을 학습시키기 위한 데이터 모음특징(Feature) : 데이터셋에서 모델이 학습할 수 있는 개별 속성레이블(label) : 예측하고자 하는 목표 변수훈련 : 모델이 데이터를 통해 학

### 각 교재별 retriever 생성부분

In [5]:
from langchain.schema import Document

# text_list를 Document 객체로 변환
documents = [Document(page_content=text) for text in text_list]


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document
import json

def get_retriever(texts: str, identifier: str):
    # text_list를 Document 객체로 변환
    documents = [Document(page_content=texts)]

    recursive_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False,
    )

    splits_recur = recursive_text_splitter.split_documents(documents)
    splits = splits_recur

    print("Top 10 chunks:")
    for i, chunk in enumerate(splits[:10], 1):
        pprint(f"\nChunk {i}:\n{chunk.page_content}")

    # OpenAI 임베딩 모델 초기화
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

    bm25_retriever = BM25Retriever.from_documents(splits)
    faiss_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

    retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, faiss_retriever],
        weights=[0.5, 0.5],  # 가중치 설정 (가중치의 합은 1.0)
    )

    return retriever


In [ ]:
ml_retriever_list = []

for idx, text in enumerate(text_list, start=1):
    identifier = f"ml_doc_{idx}"  # 고유 파일명 생성 (예: doc_1, doc_2, ...)
    retriever = get_retriever(text, identifier)
    retriever_list.append(retriever)


Top 10 chunks:
('\n'
 'Chunk 1:\n'
 '[스파르타코딩클럽] 1강. 강의 소개📘[SCC] 바닥부터 시작하는 머신러닝/📚[스파르타코딩클럽] 바닥부터 시작하는 머신러닝 - '
 '1주차/📕[스파르타코딩클럽] 1강. 강의 소개Made with📕[스파르타코딩클럽] 1강. 강의 소개[수업 목표]강의 소개 및 머신러닝 '
 '기본 개념을 알아봅시다[목차]01. 강좌 소개🤔 딥러닝이 대세라던데, 왜 머신러닝부터')
('\n'
 'Chunk 2:\n'
 '대세라던데, 왜 머신러닝부터 시작할까요?02. 머신러닝 소개강의 뒤쪽에서 나오는 어려운 문법들은 강좌 내에서 설명 후에 실습을 진행할 '
 '예정이니 걱정하지 않으셔도 됩니다!강의 뒤쪽에서 나오는 어려운 문법들은 강좌 내에서 설명 후에 실습을 진행할 예정이니 걱정하지 않으셔도 '
 '됩니다!\ufeff\u200b🤔 딥러닝이 대세라던데, 왜 머신러닝부터 시작할까요?딥러닝의 기초는 탄탄한')
('\n'
 'Chunk 3:\n'
 '시작할까요?딥러닝의 기초는 탄탄한 머신러닝 지식입니다. 기초를 튼튼히 쌓아야 딥러닝도 쉽게 이해할 수 있죠! 차근차근 기초부터 다지면, '
 '딥러닝 정복도 문제없어요! 💪02. 머신러닝 소개✔️머신러닝에 대한 기본적인 개념 및 역사등을 소개합니다1)머신러닝 소개☑️ 머신러닝의 '
 '정의컴퓨터가 명시적으로 프로그래밍 되지 않아도 데이터를 통해 학습하고, 예측할 수')
('\n'
 'Chunk 4:\n'
 '데이터를 통해 학습하고, 예측할 수 있도록 하는 기능대량의 데이터를 알고리즘에 입력하여 학습과정을 통해 모델을 생성하고 예측을 수행기존 '
 '프로그램은 명시적으로 작성된 규칙과 논리에 따라 동작한다는것에서 머신러닝과의 차이가 있습니다!기존 프로그램은 명시적으로 작성된 규칙과 '
 '논리에 따라 동작한다는것에서 머신러닝과의 차이가 있습니다!\ufeff\u200b전통적인 프로그래밍:규칙과')
('\n'
 'Chunk 5:\n'
 '프로그래밍:규칙과 논리를 프로그래머가 직접 

In [17]:
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output

In [23]:
# Prompt 및 Chain 구성
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = " ".join([doc.page_content for doc in inputs["context"]])
        print(f"Context output: {context_text}")
        return {"context": context_text, "quiz_list": inputs["quiz_list"]}

prompt = ChatPromptTemplate.from_messages([
    ("system", """
    당신은 딥러닝을 가르치는 AI 강사입니다.
    아래의 {context} 바탕으로만 한국말로 된 하나의 질문을 생성해주세요.
    (최대한 코드에 관한 시나리오적 질문이면 더 좋습니다.)
     
    [중요]
    이전에 만들었던 질문과 유사한 질문은 절대 생성하지 마세요.
    주관적인 변수에 대한 의견을 묻는 질문은 피해야 합니다.
    예를 들어, "타이타닉 데이터셋의 'embarked' 열을 숫자로 매핑할 때, 'S'는 어떤 숫자로 매핑되나요?"와 같은 질문은 피해야 합니다.

    아래의 제약 조건과 출제 방식에 맞춘 질문을 생성해주세요.
     
    제약 조건:
    1. "Context"에서 제공된 내용만 기반으로 질문을 생성하세요.
    2. AI 관련 내용이 아닌 질문은 생성하지 마세요
    3. "QuizList"에 이미 있는 질문과 유사하지 않은 새로운 질문을 생성하세요.

    출제 방식:
    - 질문은 반드시 보기가 있는 객관식(MCQ) 또는 O,X 형태로 출제하세요.
    - "Context"에 명시적으로 언급된 개념, 정의, 또는 내용을 활용하세요.
    
    Context:
    {context}
    
    QuizList:
    {quiz_list}
     
    

    """)
])

# 아래의 금지리스트들과 유사한 질문을 절대 생성하지 마시오.
#     금지리스트: {quiz_list}


In [9]:
# RAG Chain 생성 함수
def create_rag_chain(retriever):
    return (
        {
            "context": retriever,
            "quiz_list": DebugPassThrough()
        }
        | DebugPassThrough()  # DebugPassThrough()가 실제로 어떤 역할을 하는지 확인
        | ContextToText()     # Text 변환을 위한 ContextToText
        | prompt              # prompt 사용
        | llm                 # LLM 호출
        | StrOutputParser()   # 출력 파서
    )


### 동작부분

In [26]:
import random
from langchain.globals import set_llm_cache, get_llm_cache
from langchain.cache import InMemoryCache


order = int(input("선택하실 딥러닝 챕터의 교재 번호를 입력해주세요! 종료를 원하신다면 0을 입력해주세요!"))
set_llm_cache(InMemoryCache()) # 캐시 메모리 설정
llm_cache = get_llm_cache()
j = 0

while True: 

    if (order == 0):
        break

    latest_quizzes = []

    # 캐시 데이터 추출 및 확인
    if isinstance(llm_cache, InMemoryCache):
        cached_data = list(llm_cache._cache.items())  # 캐시된 모든 데이터 가져오기 (키-값 쌍)
        latest_quizzes = [str(value) for _, value in cached_data[-5:]]  # 최신 5개의 값을 문자열로 변환
        print("Latest 5 Quizzes:", latest_quizzes)
    else:
        print("현재 캐시가 InMemoryCache가 아닙니다.")

    cached_quiz_list = latest_quizzes
    cached_quiz_str = " ".join(cached_quiz_list)
    rag_chain = create_rag_chain(retriever_list[order-1])
    
    try:
        response = rag_chain.invoke(cached_quiz_str)
    except Exception as e:
        print(f"Quiz 생성 중 오류 발생: {e}")
        break
    
    quiz = response
    print(f"Cache 길이 : {len(latest_quizzes)}")
    print(f"Quiz : {quiz}")
    j+=1
    save_file(''.join(str(quiz)), f"quiz_list_{j}.txt")
    
    
    # 2. 사용자 답변 수집
    user_answer = input("답변을 입력하세요 종료를 원하시면 exit을 입력해주세요.: ").strip()

    if user_answer.strip().lower() == "exit":
        print("대화를 종료합니다.")
        break
    
    if not user_answer:
        print("답변이 비어 있습니다. 다시 입력해주세요.")
        continue

    # 3. 사용자 답변에 대한 피드백 생성
    feedback_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
    AI 강사로서 다음 퀴즈의 정답 여부를 확인하고 피드백을 제공하세요.
    피드백은 아래와 같은 형식이어야 합니다:
    
    - 정답 여부: "N번" 또는 "예/아니오"
    - 추가 설명: (정답과 관련된 추가 정보를 제공하세요)
    
    퀴즈 : {{quiz}}
    답변 : {{user_answer}}
      
    """)
    ])


    feedback_chain = feedback_prompt | llm
    feedback = feedback_chain.invoke({"quiz": quiz, "user_answer": user_answer})
    print("Feedback:")
    pprint(feedback)

Latest 5 Quizzes: []
Debug Output: 
Debug Output: {'context': [Document(metadata={}, page_content='print(f"Confusion Matrix:")\n\u200bCopyright ⓒ TeamSparta All rights reserved.'), Document(metadata={}, page_content='print(f"Confusion Matrix:")'), Document(metadata={}, page_content='숫자로 매핑합니다.\ufeff\u200b☑️ 모델 학습모델 학습 {5px}모델 학습 \ufeff\u200bPythonCopy# 모델 생성 및 학습'), Document(metadata={}, page_content=': 분류모델의 로지스틱 회귀에 대해 알아보고 실습을 통해 배워봅시다[목차]01. 로지스틱 회귀 개념02. 로지스틱 회귀분석 실습import pandas as pd'), Document(metadata={}, page_content='# 데이터 로드\ndata = load_breast_cancer()\nX = data.data\ny = data.target'), Document(metadata={}, page_content='from sklearn.datasets import load_breast_cancer\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.preprocessing import StandardScaler'), Document(metadata={}, page_content='[스파르타코딩클럽] 9강. 지도학습 : 분류모델 - 로지스틱 회귀📘[SCC] 바닥부터 시작하는 머신러닝/📚[스파르타코딩클럽] 바닥부터 시작하는 머신러닝 - 3주차/📕[스파르타코딩클럽] 9강. 지도학습 : 분류모델 - 로지스틱 회귀Made with📕[스파르타코딩클럽] 9강. 지도학습 : 분류모델 